## this gets the data into csv format

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

In [2]:
#this is commented out because it takes forever and I will just provide the filtered files. 
#however it is good to see how we did this

#open('filtered_data\q1_commit_messages_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q1_commit_messages.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q2_commit_dates_epoch_timestamp_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q2_commit_dates_epoch_timestamp.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q3_author_names_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q3_author_names.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q4_committer_names_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q4_committer_names.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q5_all_changed_files_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q5_all_changed_files.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q6_deleted_files_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q6_deleted_files.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q7_added_files_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q7_added_files.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q8_modified_files_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q8_modified_files.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q10_commit_count_by_project_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q10_commit_count_by_project.txt',encoding="utf8") if 'commit_log' in line)

## here we bring the filtered files in and merge the first set into one dataframe

In [3]:
messages = pd.read_csv('filtered_data/q1_commit_messages_filtered.txt', usecols=[0,1,2], sep='\|\|\|', names=['project_id','commit_id','message'])
timestamps = pd.read_csv('filtered_data/q2_commit_dates_epoch_timestamp_filtered.txt', usecols=[0,1,2], sep='\|\|\|', names=['project_id','commit_id','timestamp'])
authors = pd.read_csv('filtered_data/q3_author_names_filtered.txt', usecols=[0,1,2], sep='\|\|\|', names=['project_id','author_name','commit_id'])
committers = pd.read_csv('filtered_data/q4_committer_names_filtered.txt', usecols=[0,1,2], sep='\|\|\|', names=['project_id','commit_id','committer_name'])

In [4]:
#merge all the message related files together
merge1 = pd.merge(messages,timestamps, on=['project_id','commit_id'])
merge2 = pd.merge(merge1, authors, on=['project_id','commit_id'])
merged_data = pd.merge(merge2,committers, on=['project_id','commit_id'])

In [5]:
#drop any rows that dont have a message because we dont need them
merged_data.dropna(subset=['message'], inplace=True)

In [6]:
#spit out all the messages to a file alone for processing by sentistrength
merged_data.message.to_csv('sentistrength_run/messages_only.csv', header=False, index=False)

## here, we manually run the sentistrength tool on the messages_only file, which produces the messages_only_results file which we can then merge with the main dataframe

In [7]:
#bring the strengths data in from the sentistrength results file
strengths = pd.read_csv('sentistrength_run/messages_only_results.txt', usecols=[1,2], sep='\t', dtype=str, names=['positive_strength','negative_strength'])

In [8]:
#merge the strengths with the main dataframe
merged_data.reset_index(drop=True, inplace=True)
commit_messages_final = pd.concat([merged_data,strengths],axis=1)

In [9]:
#convert the timestamp to date and add a day of week field
commit_messages_final["timestamp"] = commit_messages_final["timestamp"].astype(str)
commit_messages_final["timestamp"] = commit_messages_final["timestamp"].str[:10]
commit_messages_final['timestamp'] = pd.to_datetime(commit_messages_final['timestamp'],unit='s',errors='coerce')
commit_messages_final['day_of_week'] = commit_messages_final['timestamp'].dt.weekday_name

## done with commit messages, moving on to changed files

In [10]:
#Bring in all the file related stuff
all_changes = pd.read_csv('filtered_data/q5_all_changed_files_filtered.txt', usecols=[0,1], sep='\|\|\|', names=['commit_id','filepath'])
deletes = pd.read_csv('filtered_data/q6_deleted_files_filtered.txt', usecols=[0,1], sep='\|\|\|', names=['commit_id','filepath'])
adds = pd.read_csv('filtered_data/q7_added_files_filtered.txt', usecols=[0,1], sep='\|\|\|', names=['commit_id','filepath'])
modifies = pd.read_csv('filtered_data/q8_modified_files_filtered.txt', usecols=[0,1], sep='\|\|\|', names=['commit_id','filepath'])

In [11]:
#tag the deletes
merge_deletes = pd.merge(all_changes, deletes, on=['commit_id','filepath'], how='left', indicator='modif')
merge_deletes['modif'] = np.where(merge_deletes.modif == 'both', 'Delete', '')

In [12]:
#tag the adds
merge_adds = pd.merge(all_changes, adds, on=['commit_id','filepath'], how='left', indicator='modif')
merge_adds['modif'] = np.where(merge_adds.modif == 'both', 'Add', '')

In [13]:
#tag the modifies
merge_modifies = pd.merge(all_changes, modifies, on=['commit_id','filepath'], how='left', indicator='modif')
merge_modifies['modif'] = np.where(merge_modifies.modif == 'both', 'Modify', '')

In [14]:
#merge the deletes and adds tag frames together
temp_files_merge = pd.merge(merge_deletes, merge_adds, on=['commit_id','filepath'])

In [15]:
#squash down the columns in the delete/add frame so the tags are in the same column
temp_files_merge['modif'] = temp_files_merge['modif_x'].where(temp_files_merge['modif_x'] != '', temp_files_merge['modif_y'])
temp_files_merge = temp_files_merge.drop(['modif_x','modif_y'],axis=1)

In [16]:
#merge the delete/add frame and the modify frame together
temp_files_merge2 = pd.merge(temp_files_merge, merge_modifies, on=['commit_id','filepath'])

In [17]:
#squash down the columns in the final frame so the all tags are in the same column
temp_files_merge2['modification'] = temp_files_merge2['modif_x'].where(temp_files_merge2['modif_x'] != '', temp_files_merge2['modif_y'])
files_changed_final = temp_files_merge2.drop(['modif_x','modif_y'],axis=1)

##  here are the final pandas dataframes:

In [18]:
commit_messages_final.head()

,project_id,commit_id,message,timestamp,author_name,committer_name,positive_strength,negative_strength,day_of_week
0,10000244.0,0515c7d89f97bf7f1e488ac39dcc8fe1ee79f605,photo in progress,2013-05-11 15:52:20,syhw,syhw,1,-1,Saturday
1,10000244.0,07abeaf9bd85eb77f0774607161a7ccb3ed8dff3,stashing,2013-05-12 09:29:50,syhw,syhw,1,-1,Sunday
2,10000244.0,11f46d585bfc263e11b0556566559e6513cca0e4,R,2013-05-12 12:10:17,syhw,syhw,1,-1,Sunday
3,10000244.0,25ffeb626cd9146c483c6144b8208b5bd6cf26c4,"merged photo (working) with master, see TODOs...",2013-05-12 07:18:18,syhw,syhw,1,-1,Sunday
4,10000244.0,33ac9b6d9da4234c93ae3d9e0d7b0f41edb99d8a,Request task,2013-05-12 10:01:57,syhw,syhw,1,-1,Sunday


In [19]:
files_changed_final.head()

,commit_id,filepath,modification
0,000e3673387b662e32f1037eeaea2aa29b630448,Sources/Internal/Platform/TemplateMacOS/MainW...,Modify
1,000e3673387b662e32f1037eeaea2aa29b630448,Tools/ResourceEditor/Classes/Qt/Tools/LodCorr...,Delete
2,000e3673387b662e32f1037eeaea2aa29b630448,Tools/ResourceEditor/Classes/Qt/Tools/LodCorr...,Delete
3,000e3673387b662e32f1037eeaea2aa29b630448,Tools/ResourceEditor/Classes/Qt/Tools/LodCorr...,Delete
4,000e3673387b662e32f1037eeaea2aa29b630448,Tools/ResourceEditor/Classes/Qt/Tools/LodCorr...,Delete


# Analysis

### RQ1

In [20]:
#compute final sentiment value from sum of positive and negative
commit_messages_final['final_sentiment'] = pd.to_numeric(commit_messages_final['positive_strength']) + pd.to_numeric(commit_messages_final['negative_strength'])
commit_messages_final

,project_id,commit_id,message,timestamp,author_name,committer_name,positive_strength,negative_strength,day_of_week,final_sentiment
0,10000244.0,0515c7d89f97bf7f1e488ac39dcc8fe1ee79f605,photo in progress,2013-05-11 15:52:20,syhw,syhw,1,-1,Saturday,0
1,10000244.0,07abeaf9bd85eb77f0774607161a7ccb3ed8dff3,stashing,2013-05-12 09:29:50,syhw,syhw,1,-1,Sunday,0
2,10000244.0,11f46d585bfc263e11b0556566559e6513cca0e4,R,2013-05-12 12:10:17,syhw,syhw,1,-1,Sunday,0
3,10000244.0,25ffeb626cd9146c483c6144b8208b5bd6cf26c4,"merged photo (working) with master, see TODOs...",2013-05-12 07:18:18,syhw,syhw,1,-1,Sunday,0
4,10000244.0,33ac9b6d9da4234c93ae3d9e0d7b0f41edb99d8a,Request task,2013-05-12 10:01:57,syhw,syhw,1,-1,Sunday,0
5,10000244.0,353d2a5b6f913e5b538b8bbc784c41f6fe85feb0,Merge branch 'master' of github.com:SnippyHol...,2013-05-12 11:46:06,syhw,syhw,1,-1,Sunday,0
6,10000244.0,4270f596682bf83e51a9480bb60821e33a15a9c7,first import resto photo,2013-05-11 14:10:16,syhw,syhw,1,-1,Saturday,0
7,10000244.0,4a1eb322629c75e5ecddeb4afed63f9ecee57ced,remove TakePic,2013-05-12 08:38:14,syhw,syhw,1,-1,Sunday,0
8,10000244.0,500a023f5b83290f0c4aafa97be6660b4d9e5c2a,OCR added,2013-05-12 11:10:22,Vincent Delaitre,Vincent Delaitre,1,-1,Sunday,0
9,10000244.0,52c2b94a120cc7ef5d777c9f6565d3e0c1167196,try readme,2013-05-12 09:15:28,syhw,syhw,1,-1,Sunday,0


In [21]:
#count the number of commits for each sentiment level
commits_by_sentiment = commit_messages_final.groupby(['final_sentiment']).count().reset_index()
commits_by_sentiment = commits_by_sentiment[['final_sentiment','project_id']]
commits_by_sentiment.rename(columns={'project_id':'count'}, inplace=True)

In [22]:
commits_by_sentiment

,final_sentiment,count
0,-4,47
1,-3,1819
2,-2,31340
3,-1,262278
4,0,1821461
5,1,125868
6,2,9911
7,3,295
8,4,6


In [23]:
commits_by_project = commit_messages_final.groupby(['project_id']).count().reset_index()

In [24]:
commits_by_project = commits_by_project[['project_id','commit_id']]
commits_by_project.rename(columns={'commit_id':'count'}, inplace=True)
commits_by_project.sort_values(['count'], inplace=True)

In [25]:
commits_by_project.to_csv('commits_by_project_sorted.csv', sep='\t', encoding='utf-8')

### manually obtained large, average and low groups by examining file

In [26]:
largelist = ['12496978','10613094','5153143','7785050','1968812']
avglist = ['2424377','5256179','365893','6719407','13010741']
lowlist = ['10453653','10530838','11416657','4067771','1571039']

In [27]:
large_projects = commit_messages_final.loc[commit_messages_final['project_id'].isin(largelist)]
avg_projects = commit_messages_final.loc[commit_messages_final['project_id'].isin(avglist)]
low_projects = commit_messages_final.loc[commit_messages_final['project_id'].isin(lowlist)]

### RQ2

### RQ3
Is there a correlation between the number of changed files and developer sentiment? 

In [30]:
files_changed_count = files_changed_final.groupby('commit_id').count().reset_index()
files_changed_count = files_changed_count[['commit_id', 'modification']]
files_changed_count.head()

,commit_id,modification
0,000e3673387b662e32f1037eeaea2aa29b630448,13
1,000e722626473ce76fcc2a604b1ef8ba7860ef7e,2
2,0013e3a3a6ec30cd05d17b1d9597617866254d43,1
3,00196d38b331102892a87d0c33fa5f7d9f6d3958,1
4,001a91d48e1cbaaf3f139f73c06e68e95008b160,8


In [31]:
files_changed_count.shape[0] 

14956

In [34]:
commit_messages_final.drop_duplicates(subset=['commit_id'], keep='first').shape[0]

1925546

In [35]:
#files_changed_count = pd.merge(files_changed_count,commit_messages_final[['final_sentiment']],on='commit_id')
files_changed_count.head()


,commit_id,modification
0,000e3673387b662e32f1037eeaea2aa29b630448,13
1,000e722626473ce76fcc2a604b1ef8ba7860ef7e,2
2,0013e3a3a6ec30cd05d17b1d9597617866254d43,1
3,00196d38b331102892a87d0c33fa5f7d9f6d3958,1
4,001a91d48e1cbaaf3f139f73c06e68e95008b160,8
